In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import os

from time import sleep

# Search tweets

In [ ]:
count = 0
tema = "copa"

print("O tema escolhido foi:", tema)
for ano in range(2016, 2018):
    for mes in range(1, 13):
        if count < 9:
            count += 1
            continue
        print("Vamos pegar os tweets do mês", mes, "do ano", ano)
        os.system('scrape-twitter search --query "%s ? since:%i-0%i-01 until:2016-0%i-01 lang:pt -filter:retweets -filter:links" --type latest > perguntas%i-%i.json'%(tema, ano, mes, mes+1, ano, mes))
        print("Processo terminado! - aguardando 60s para começar o próximo mês \n")
        sleep(60)

# Carregando o que salvamos
### 

In [ ]:
data = json.load(open('copa/perguntas2016-1.json'))
df = pd.DataFrame(data)

df.head()

# Tratando a informação

In [ ]:
df = df[df.replyCount > 0]
df = df[df.isReplyTo == False]
#df = df[df.userMentions == []]
df = df.assign(answer=None)

len(df)
df.tail()

# Search answers

In [ ]:
for index, row in df.iterrows():
    username = row["screenName"]
    tweet_id = row["id"]
    #os.system("scrape-twitter conversation %s %s >> respostas.json" % (username, tweet_id))
    try:
        data = json.loads(os.popen("scrape-twitter conversation %s %s" % (username, tweet_id)).read())
        print("Não deu merda!")
    except:
        print("Erro!")       
    try:
        df.set_value(index, 'answer', data[1]['text'])
    except:
        print("Index problem: ignoring row")

# Tratando os tweets

In [ ]:
df = df[['text', 'answer']]
df = df.dropna()

In [ ]:
qa = [{"q": x, "a": y} for x, y in df.values]
q = np.array(df["text"])
a = np.array([df["answer"], 280])

### Como resultado temos *qa* como uma lista de dicionários, onde q é a pergunta e a é a resposta